# High-level TF Example

In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
from common.params_lstm import *
from common.utils import *

In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tf.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.1
Tensorflow:  1.4.0
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [3]:
EPOCHS = 50
BATCHSIZE = 32
N_CLASSES = 10
N_STEPS = 32
N_INPUTS = 32*3
N_HIDDEN = 64  # RNN units 

In [4]:
def create_symbol(X, n_steps=N_STEPS, nhid=N_HIDDEN, n_classes=N_CLASSES):
    # Convert x to a list[max_time] where element has shape=2 [batch_size, depth]
    x = tf.unstack(X, n_steps, axis=1)
    cell = tf.nn.rnn_cell.BasicRNNCell(nhid)
    outputs, states = tf.contrib.rnn.static_rnn(cell, x, dtype=tf.float32)
    logits = tf.layers.dense(outputs[-1], n_classes, activation=None, name='output')
    return logits

In [5]:
def init_model(m, y, lr=LR, b1=BETA_1, b2=BETA_2, eps=EPS):
    # Single-class labels, don't need dense one-hot
    # Expects unscaled logits, not output of tf.nn.softmax
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=m, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer(lr, b1, b2, eps)
    training_op = optimizer.minimize(loss)
    return training_op

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=False)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 32, 32, 3) (10000, 32, 32, 3) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 668 ms, sys: 593 ms, total: 1.26 s
Wall time: 1.26 s


In [7]:
# Sequences of 32 time-steps, each containing 32*3 units
x_train = x_train.reshape(x_train.shape[0], N_STEPS, N_INPUTS)
x_test = x_test.reshape(x_test.shape[0], N_STEPS, N_INPUTS)
print(x_train.shape, x_test.shape)

(50000, 32, 96) (10000, 32, 96)


In [8]:
%%time
# Place-holders
X = tf.placeholder(tf.float32, shape=[None, N_STEPS, N_INPUTS])
y = tf.placeholder(tf.int32, shape=[None])  # Sparse
# Initialise model
sym = create_symbol(X)

CPU times: user 917 ms, sys: 91.1 ms, total: 1.01 s
Wall time: 1.01 s


In [9]:
%%time
model = init_model(sym, y)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# Accuracy logging
correct = tf.nn.in_top_k(sym, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

CPU times: user 724 ms, sys: 702 ms, total: 1.43 s
Wall time: 1.43 s


In [10]:
%%time
# Main training loop: 57s
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        sess.run(model, feed_dict={X: data, y: label})
    # Accuracy
    acc_train = sess.run(accuracy, feed_dict={X: x_train, y: y_train})
    print(j, "Train accuracy:", acc_train)

0 Train accuracy: 0.31296
1 Train accuracy: 0.3248
2 Train accuracy: 0.33898
3 Train accuracy: 0.34426
4 Train accuracy: 0.29618
5 Train accuracy: 0.34878
6 Train accuracy: 0.33542
7 Train accuracy: 0.35038
8 Train accuracy: 0.34348
9 Train accuracy: 0.32752
10 Train accuracy: 0.29506
11 Train accuracy: 0.31352
12 Train accuracy: 0.3234
13 Train accuracy: 0.3488
14 Train accuracy: 0.28352
15 Train accuracy: 0.29558
16 Train accuracy: 0.29612
17 Train accuracy: 0.26252
18 Train accuracy: 0.3023
19 Train accuracy: 0.3002
20 Train accuracy: 0.31234
21 Train accuracy: 0.3139
22 Train accuracy: 0.3065
23 Train accuracy: 0.29508
24 Train accuracy: 0.32046
25 Train accuracy: 0.26174
26 Train accuracy: 0.29402
27 Train accuracy: 0.3122
28 Train accuracy: 0.29226
29 Train accuracy: 0.31272
30 Train accuracy: 0.28938
31 Train accuracy: 0.30104
32 Train accuracy: 0.2955
33 Train accuracy: 0.30156
34 Train accuracy: 0.24114
35 Train accuracy: 0.28714
36 Train accuracy: 0.29512
37 Train accuracy: 0

In [13]:
%%time
# Main evaluation loop: 3.22s
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    pred = tf.argmax(sym,1)
    output = sess.run(pred, feed_dict={X: data})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = output
    c += 1

CPU times: user 8.92 s, sys: 589 ms, total: 9.5 s
Wall time: 9.29 s


In [14]:
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.31099759615384615
